Retrieving data from Axelbib
===========================
At this stage the data are within the django data base Axelbib. There they have been 
cleaned and missing information has been manually filled in. 

Axelbib has a django rest framework. 
The data will be retrieved by the rest framework in a json format
and will be push into a pandas dataframe following the Arcas structure.

In [1]:
import requests
import pandas as pd
from itertools import product
import collections
import json
from pandas.io.json import json_normalize

In [2]:
from password import password

In [3]:
keys = ['title', 'provenance', 'score', 'read', 'unique_key',
        'date', 'abstract', 'author', 'journal', 'labels',
        'key_word', 'list_strategies', 'pages', 'key']

In [4]:
def raw_to_db(an_article):
    an_article['author'] = [f['name'] for f in an_article['author']]
    an_article['key_word'] = [f['key_word'] for f in an_article['key_word']]
    an_article['date'] = an_article['date']['year']
    an_article['labels'] = [f['label'] for f in an_article['labels']]
    
    for _, field in enumerate(an_article):
        if not an_article[field]:
            an_article[field] = ['None']
   
    values = []
    for key in keys:
        if type(an_article[key]) is not list:
            values.append([an_article[key]])
        else:
            values.append(an_article[key])
    data = []
    for row in product(*values):
        data.append(row)
    df = pd.DataFrame(data, columns=keys)    
    return df

In [5]:
page = 1
status = 200
dfs = []
while page <= 115:
    req = requests.get('http://127.0.0.1:8000/article/?page={}'.format(page), auth=("nikoleta", password))
    status =  req.status_code
    page += 1
    results_of_req = req.json()['results']
    for raw_article in results_of_req:
        df = raw_to_db(raw_article)
        dfs.append(df)        

In [6]:
df = pd.concat(dfs, ignore_index=True)

In [7]:
len(dfs), len(df), len(df.groupby("unique_key").count())

(1145, 11034, 1145)

In [8]:
df.to_json('data.json')